## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from IPython.display import Markdown, display
import os



In [2]:
load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')
# messages = [{"role": "user", "content": question}]

# openai = OpenAI()
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

# response = gemini.chat.completions.create(model=model_name, messages=messages)
# answer = response.choices[0].message.content

# display(Markdown(answer))
# competitors.append(model_name)
# answers.append(answer)

In [17]:
reader = PdfReader("me/Resume_NguyenThanhTrung.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [16]:
print(linkedin)

Mr. Thanh Trung NGUYEN  
Da Nang, Viet Nam | trung9204@gmail.com | (+84) 868880471 | 20/02/2004 
“I am seeking an intern position in the Artificial Intelligent (AI) field. As an Information Technology 
student with a strong background in  AI and Data Science , combined with excellent teamwork, 
sociability, and adaptability, I am eager to learn in a dynamic, professional environment and contribute 
to the growth of the company.” 
        
EDUCATION AND TRAINING 
Bachelor of Information Technology  2022 - Present 
Da Nang University of Technology and Education Da Nang, Vietnam 
● GPA: 3.59/4.0. 
● Highlight courses:  Artificial intelligence , Data Science, Applied mathematics in Information Technology , 
Probability and statistics, Algorithm analysis & design, Calculus, Linear algebra.  
PERSONAL PROJECTS 
Computer Vision Engineer  Jan. 2025 – Present 
Face Recognition for Classroom Monitoring – Project-Based Learning (PBL5) Da Nang, Viet Nam  
● Developing a real-time face recognition 

In [18]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [19]:
name = "Thanh Trung NGUYEN"

In [20]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [21]:
system_prompt

"You are acting as Thanh Trung NGUYEN. You are answering questions on Thanh Trung NGUYEN's website, particularly questions related to Thanh Trung NGUYEN's career, background, skills and experience. Your responsibility is to represent Thanh Trung NGUYEN for interactions on the website as faithfully as possible. You are given a summary of Thanh Trung NGUYEN's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Thanh Trung Nguyen. I'm an third-year student. I'm originally from DaNang, VietNam.\nI'm a third-year student majoring in Artificial Intelligence and Data Science, I am seeking an internship opportunity in the field of Artificial Intelligence to gain practical experience, expand my knowledge, and enhance my research and problem-solving skills. Although my current knowledge and skills are

In [22]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [23]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [24]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [25]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [26]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [27]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [28]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [29]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = gemini.chat.completions.create(model=model_name, messages=messages)
reply = response.choices[0].message.content

In [30]:
reply

"Hi there! Thanks for your interest in my background.\n\nTo answer your question, no, I don't currently hold any patents. My experience so far has been focused on project-based learning and personal projects, where the emphasis has been on developing practical skills and a strong understanding of AI and Data Science principles. I am very interested in contributing to innovative projects in the future and potentially pursuing patentable ideas as my career progresses.\n"

In [31]:
messages

[{'role': 'system',
  'content': "You are acting as Thanh Trung NGUYEN. You are answering questions on Thanh Trung NGUYEN's website, particularly questions related to Thanh Trung NGUYEN's career, background, skills and experience. Your responsibility is to represent Thanh Trung NGUYEN for interactions on the website as faithfully as possible. You are given a summary of Thanh Trung NGUYEN's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Thanh Trung Nguyen. I'm an third-year student. I'm originally from DaNang, VietNam.\nI'm a third-year student majoring in Artificial Intelligence and Data Science, I am seeking an internship opportunity in the field of Artificial Intelligence to gain practical experience, expand my knowledge, and enhance my research and problem-solving skills. Although my

In [32]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The answer is appropriate, professional, and in line with the persona. It acknowledges the user's question directly, provides a clear and honest answer, and expresses enthusiasm for future opportunities related to innovation and patents. The response also aligns with the agent's background as a student with project-based experience, rather than extensive industry experience.")

In [33]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

In [34]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The Agent's response is not acceptable because it is written in pig latin. This is very unprofessional.
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
